In [1]:
import numpy as np
import h5py 

In [2]:
trajectory_file = h5py.File('LEFPositions.h5', mode='r')

LEFNum = trajectory_file.attrs["LEFNum"]  # number of LEFs
LEFpositions = trajectory_file["positions"]  # array of LEF positions  

steps = 100 # MD steps per step of cohesin  (set to ~800 in real sims)

In [3]:
Nframes = LEFpositions.shape[0] # length of the saved trajectory (>25000 in real sims)
print(f'Length of the saved trajectory: {Nframes}')

Length of the saved trajectory: 200


In [4]:
block = 0  # starting block 

# new parameters because some things changed 
saveEveryBlocks = 10   # save every 10 blocks (saving every block is now too much almost)
restartSimulationEveryBlocks = 100

# parameters for smc bonds
smcBondWiggleDist = 0.2
smcBondDist = 0.5

# assertions for easy managing code below 
assert (Nframes % restartSimulationEveryBlocks) == 0 
assert (restartSimulationEveryBlocks % saveEveryBlocks) == 0

savesPerSim = restartSimulationEveryBlocks // saveEveryBlocks
simInitsTotal  = (Nframes) // restartSimulationEveryBlocks

# Loop extrusion bond updater

In [5]:
class bondUpdater(object):

    def __init__(self, LEFpositions):
        """
        :param smcTransObject: smc translocator object to work with
        """
        self.LEFpositions = LEFpositions
        self.curtime  = 0
        self.allBonds = []

    def setParams(self, activeParamDict, inactiveParamDict):
        """
        A method to set parameters for bonds.
        It is a separate method because you may want to have a Simulation object already existing

        :param activeParamDict: a dict (argument:value) of addBond arguments for active bonds
        :param inactiveParamDict:  a dict (argument:value) of addBond arguments for inactive bonds

        """
        self.activeParamDict = activeParamDict
        self.inactiveParamDict = inactiveParamDict


    def setup(self, bondForce,  blocks=100, smcStepsPerBlock=1):
        """
        A method that milks smcTranslocator object
        and creates a set of unique bonds, etc.

        :param bondForce: a bondforce object (new after simulation restart!)
        :param blocks: number of blocks to precalculate
        :param smcStepsPerBlock: number of smcTranslocator steps per block
        :return:
        """


        if len(self.allBonds) != 0:
            raise ValueError("Not all bonds were used; {0} sets left".format(len(self.allBonds)))

        self.bondForce = bondForce

        #precalculating all bonds
        allBonds = []
        
        loaded_positions  = self.LEFpositions[self.curtime : self.curtime+blocks]
        allBonds = [[(int(loaded_positions[i, j, 0]), int(loaded_positions[i, j, 1])) 
                        for j in range(loaded_positions.shape[1])] for i in range(blocks)]

        self.allBonds = allBonds
        self.uniqueBonds = list(set(sum(allBonds, [])))

        #adding forces and getting bond indices
        self.bondInds = []
        self.curBonds = allBonds.pop(0)

        for bond in self.uniqueBonds:
            paramset = self.activeParamDict if (bond in self.curBonds) else self.inactiveParamDict
            ind = bondForce.addBond(bond[0], bond[1], **paramset) # changed from addBond
            self.bondInds.append(ind)
        self.bondToInd = {i:j for i,j in zip(self.uniqueBonds, self.bondInds)}
        
        self.curtime += blocks 
        
        return self.curBonds,[]


    def step(self, context, verbose=False):
        """
        Update the bonds to the next step.
        It sets bonds for you automatically!
        :param context:  context
        :return: (current bonds, previous step bonds); just for reference
        """
        if len(self.allBonds) == 0:
            raise ValueError("No bonds left to run; you should restart simulation and run setup  again")

        pastBonds = self.curBonds
        self.curBonds = self.allBonds.pop(0)  # getting current bonds
        bondsRemove = [i for i in pastBonds if i not in self.curBonds]
        bondsAdd = [i for i in self.curBonds if i not in pastBonds]
        bondsStay = [i for i in pastBonds if i in self.curBonds]
        if verbose:
            print("{0} bonds stay, {1} new bonds, {2} bonds removed".format(len(bondsStay),
                                                                            len(bondsAdd), len(bondsRemove)))
        bondsToChange = bondsAdd + bondsRemove
        bondsIsAdd = [True] * len(bondsAdd) + [False] * len(bondsRemove)
        for bond, isAdd in zip(bondsToChange, bondsIsAdd):
            ind = self.bondToInd[bond]
            paramset = self.activeParamDict if isAdd else self.inactiveParamDict
            self.bondForce.setBondParameters(ind, bond[0], bond[1], **paramset)  # actually updating bonds
        self.bondForce.updateParametersInContext(context)  # now run this to update things in the context
        return self.curBonds, pastBonds
    

milker = bondUpdater(LEFpositions)

In [6]:
density = 0.2  # density of the PBC box
num_chains = 3  # simulation uses some equivalent chains  (5 in a real sim)

In [7]:
# compartment labels
oneChainMonomerTypes = np.zeros(100).astype(int)
oneChainMonomerTypes[:20] = 1
oneChainMonomerTypes[30:40] = 2

monomerTypes = np.tile(oneChainMonomerTypes, num_chains)
N_chain = len(oneChainMonomerTypes)  
N = len(monomerTypes)

print(f'N_chain: {N_chain}')  # ~8000 in a real sim
print(f'N: {N}')   # ~40000 in a real sim

assert N == trajectory_file.attrs["N"]

N_chain: 100
N: 300


In [8]:
import polychrom
from polychrom.starting_conformations import grow_cubic
from polychrom.hdf5_format import HDF5Reporter, list_URIs, load_URI, load_hdf5_file
from polychrom.simulation import Simulation
from polychrom import polymerutils
from polychrom import forces
from polychrom import forcekits
import time

In [9]:
data = grow_cubic(N,int((N/(density*1.2))**0.333))  # starting conformation
reporter = HDF5Reporter(folder='conformations/', max_data_length=50)

PBC_width = (N/density)**0.333

# create chains
chains = [(N_chain*(k),N_chain*(k+1),0) for k in range(num_chains)]

# create interaction matrix
interactionMatrix = np.array([[0, 0, 0], [0, 0.2, 0.2], [0, 0.2, 0.4]])
print(interactionMatrix)
attraction_radius = 1.5

[[0.  0.  0. ]
 [0.  0.2 0.2]
 [0.  0.2 0.4]]


# Run the simulation

In [10]:
for iteration in range(simInitsTotal):
    a = Simulation(N=N, 
                   error_tol=0.01, 
                   collision_rate=0.01, 
                   integrator ="variableLangevin", 
                   platform="CPU", 
                   PBCbox=(PBC_width, PBC_width, PBC_width),
                   reporters=[reporter])
    a.set_data(data)
    a.add_force(
        polychrom.forcekits.polymer_chains(
            a,
            chains=chains,
            nonbonded_force_func=polychrom.forces.heteropolymer_SSW,
            nonbonded_force_kwargs={
                'attractionEnergy': 0,  # base attraction energy for all monomers
                'attractionRadius': attraction_radius,
                'interactionMatrix': interactionMatrix,
                'monomerTypes': monomerTypes,
                'extraHardParticlesIdxs': []
            },
            bond_force_kwargs={
                'bondLength': 1,
                'bondWiggleDistance': 0.05
            },
            angle_force_kwargs={
                'k': 1.5
            }
        )
    )
    
    # ------------ initializing milker; adding bonds ---------
    # copied from addBond
    kbond = a.kbondScalingFactor / (smcBondWiggleDist ** 2)
    bondDist = smcBondDist * a.length_scale

    activeParams = {"length":bondDist,"k":kbond}
    inactiveParams = {"length":bondDist, "k":0}
    milker.setParams(activeParams, inactiveParams)
     
    # this step actually puts all bonds in and sets first bonds to be what they should be
    milker.setup(bondForce=a.force_dict['harmonic_bonds'],
                blocks=restartSimulationEveryBlocks)

    # If your simulation does not start, consider using energy minimization below
    if iteration == 0:
        a.local_energy_minimization() 
    else:
        a._apply_forces()
    
    for i in range(restartSimulationEveryBlocks):        
        if i % saveEveryBlocks == (saveEveryBlocks - 1):  
            a.do_block(steps=steps)
        else:
            a.integrator.step(steps)  # do steps without getting the positions from the GPU (faster)
        if i < restartSimulationEveryBlocks - 1: 
            curBonds, pastBonds = milker.step(a.context)  # this updates bonds. You can do something with bonds here
    data = a.get_data()  # save data and step, and delete the simulation
    del a
    
    reporter.blocks_only = True  # Write output hdf5-files only for blocks
    
    time.sleep(0.2)  # wait 200ms for sanity (to let garbage collector do its magic)

reporter.dump_data()

INFO:root:Performing local energy minimization
INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:Using periodic boundary conditions
INFO:root:adding force heteropolymer_SSW 2
INFO:root:Particles loaded. Potential energy is 3.763063
INFO:root:before minimization eK=1.4037053071720857, eP=3.7630626486460526, time=0.0 ps


Exclude neighbouring chain particles from heteropolymer_SSW
Number of exceptions: 297


INFO:root:Particles loaded. Potential energy is 0.496655
INFO:root:after minimization eK=1.4037053071720857, eP=0.37554622415840055, time=0.0 ps
INFO:root:block    0 pos[1]=[4.8 1.1 13.5] dr=4.93 t=97.2ps kin=4.42 pot=3.03 Rg=5.719 SPS=1000 dt=96.2fs dx=45.15pm 
INFO:root:block    1 pos[1]=[4.4 0.2 9.6] dr=4.28 t=193.1ps kin=3.43 pot=2.91 Rg=5.567 SPS=891 dt=95.9fs dx=39.65pm 
INFO:root:block    2 pos[1]=[5.1 -0.3 11.5] dr=4.82 t=292.1ps kin=2.40 pot=2.38 Rg=5.668 SPS=973 dt=106.4fs dx=36.84pm 
INFO:root:block    3 pos[1]=[3.6 -2.3 10.5] dr=4.48 t=398.5ps kin=2.40 pot=2.04 Rg=5.348 SPS=1067 dt=106.4fs dx=36.86pm 
INFO:root:block    4 pos[1]=[6.6 10.2 9.6] dr=4.90 t=504.9ps kin=2.94 pot=2.02 Rg=5.809 SPS=982 dt=106.4fs dx=40.77pm 
INFO:root:block    5 pos[1]=[4.2 9.7 5.9] dr=4.34 t=610.0ps kin=3.05 pot=2.45 Rg=5.597 SPS=980 dt=103.7fs dx=40.45pm 
INFO:root:block    6 pos[1]=[6.2 4.2 4.9] dr=5.06 t=713.6ps kin=2.66 pot=2.38 Rg=5.517 SPS=1032 dt=103.7fs dx=37.76pm 
INFO:root:block    7 po

Exclude neighbouring chain particles from heteropolymer_SSW
Number of exceptions: 297


INFO:root:block    0 pos[1]=[-0.3 -1.0 7.6] dr=4.69 t=110.8ps kin=2.32 pot=2.02 Rg=5.862 SPS=1100 dt=109.9fs dx=37.40pm 
INFO:root:block    1 pos[1]=[2.0 0.2 6.1] dr=5.39 t=220.7ps kin=2.28 pot=1.97 Rg=5.538 SPS=887 dt=109.9fs dx=37.08pm 
INFO:root:block    2 pos[1]=[5.2 -2.8 4.7] dr=5.03 t=330.6ps kin=2.08 pot=1.98 Rg=5.548 SPS=929 dt=109.9fs dx=35.41pm 
INFO:root:block    3 pos[1]=[9.0 -4.4 7.6] dr=4.13 t=440.5ps kin=1.89 pot=1.64 Rg=5.931 SPS=1004 dt=109.9fs dx=33.75pm 
INFO:root:block    4 pos[1]=[7.9 -5.2 8.9] dr=3.94 t=553.5ps kin=1.51 pot=1.70 Rg=5.444 SPS=1054 dt=118.7fs dx=32.62pm 
INFO:root:block    5 pos[1]=[4.0 -6.4 8.3] dr=4.21 t=672.1ps kin=1.56 pot=1.57 Rg=5.526 SPS=1013 dt=118.7fs dx=33.13pm 
INFO:root:block    6 pos[1]=[5.8 -2.7 11.7] dr=4.48 t=790.8ps kin=1.89 pot=1.60 Rg=5.459 SPS=1031 dt=118.7fs dx=36.38pm 
INFO:root:block    7 pos[1]=[5.9 -4.9 14.2] dr=4.33 t=908.0ps kin=2.02 pot=1.78 Rg=5.414 SPS=1178 dt=114.5fs dx=36.35pm 
INFO:root:block    8 pos[1]=[6.6 -2.3 11

# Visualize a conformation

In [11]:
import nglutils.nglutils as ngu

_ColormakerRegistry()

In [12]:
files = list_URIs('conformations/')
files

['conformations/blocks_0-19.h5::0',
 'conformations/blocks_0-19.h5::1',
 'conformations/blocks_0-19.h5::2',
 'conformations/blocks_0-19.h5::3',
 'conformations/blocks_0-19.h5::4',
 'conformations/blocks_0-19.h5::5',
 'conformations/blocks_0-19.h5::6',
 'conformations/blocks_0-19.h5::7',
 'conformations/blocks_0-19.h5::8',
 'conformations/blocks_0-19.h5::9',
 'conformations/blocks_0-19.h5::10',
 'conformations/blocks_0-19.h5::11',
 'conformations/blocks_0-19.h5::12',
 'conformations/blocks_0-19.h5::13',
 'conformations/blocks_0-19.h5::14',
 'conformations/blocks_0-19.h5::15',
 'conformations/blocks_0-19.h5::16',
 'conformations/blocks_0-19.h5::17',
 'conformations/blocks_0-19.h5::18',
 'conformations/blocks_0-19.h5::19']

In [13]:
data = load_URI(files[19])
atom_names = [chr(a+65) for a in monomerTypes]

pos_in_box = data['pos']
pos_in_box = pos_in_box % PBC_width
pos_in_box = pos_in_box / 5

top = ngu.mdtop_for_polymer(N, atom_names=atom_names, chains=chains, exclude_bonds=[k for k in range(N)])

In [14]:
view = ngu.xyz2nglview(pos_in_box, top=top)
view

NGLWidget()

In [15]:
view.clear_representations()
ngu.rep_add.uniform(view, 0xff0000, ':.A')
ngu.rep_add.uniform(view, 0x00ff00, ':.B')
ngu.rep_add.uniform(view, 0x0000ff, ':.C')